#  Using Azure Blob Storage with AzureClusterlessHPC

This notebook demonstrates how to interact with Azure Blob storage when executing a remote function via AzureClusterlessHPC.

We start with loading AzureClusterlessHPC and creating a pool. Notice that in the pool startup script, we have to install pip3 and the Python SDKs for blob and queue storage (see "pool_startup_script.sh" for details).

In [1]:
# Install required Julia packages
using Pkg
Pkg.add("PyCall")

# Set path to credentials
ENV["CREDENTIALS"] = joinpath(pwd(), "credentials.json")

# Set path to batch parameters (pool id, VM types, etc.)
ENV["PARAMETERS"] = joinpath(pwd(), "parameters.json")

# Load package
using AzureClusterlessHPC;

In [2]:
# Create pool
startup_script = "pool_startup_script.sh"
create_pool_and_resource_file(startup_script);

Pool 1 of 1 in canadacentral already exists.


Next, we load PyCall and the blob storage module:

In [3]:
@batchdef using PyCall
@batchdef azureblob = pyimport("azure.storage.blob");

Then we define our function that we execute remotely through Azure Batch. Within the function, we create an Azure Blob client using the Blob Python SDK. The blob client can be used to create new blobs or download blobs from an account. In our example function, we simply list the current containers in our storage account. (Before running this cell, fill in your storage credentials for the account that you want to use.)

In [4]:
@batchdef function hello_world()
    
    # Create blob client
    blob_client = azureblob.BlockBlobService(
        account_name = "storageaccountname",
        account_key = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
    )

    # List current containers
    output = blob_client.list_containers().items
    print(output)
    
    return output
end;

We execute the function remotely using the @batchexec macro:

In [5]:
bctrl = @batchexec hello_world();

In [6]:
output = fetch(bctrl)
print(output);

Monitoring tasks for 'Completed' state, timeout in 60 minutes ...Creating job [BatchJob_5Ztj47jz_1]...
Uploading file /home/pwitte/.julia/dev/AzureClusterlessHPC/src/runtime/application-cmd to blob container [azureclusterlesstemp]...
Uploading file /home/pwitte/.julia/dev/AzureClusterlessHPC/src/runtime/batch_runtime.jl to blob container [azureclusterlesstemp]...
Uploading file packages.dat to container [azureclusterlesstemp]...
Uploading file task_1.dat to container [azureclusterlesstemp]...
..
Fetch output from task task_1
PyObject[PyObject <azure.storage.blob.models.Container object at 0x7f926bd267f0>, PyObject <azure.storage.blob.models.Container object at 0x7f926bd26828>, PyObject <azure.storage.blob.models.Container object at 0x7f9276abf908>, PyObject <azure.storage.blob.models.Container object at 0x7f9276abf860>, PyObject <azure.storage.blob.models.Container object at 0x7f9276ad9a90>, PyObject <azure.storage.blob.models.Container object at 0x7f926bd26710>, PyObject <azure.storag

Clean up resources:

In [7]:
destroy!(bctrl);

# Copyright

Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License (MIT). See LICENSE in the repo root for license information.